In [1]:
import random
import numpy as np
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import torch
import learn2learn as l2l
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import StepLR
# The corase reconstruction is the rss of the zerofilled multi-coil kspaces
# after inverse FT.
from functions.data.transforms import UnetDataTransform, UnetDataTransform_norm, normalize
# Import a torch.utils.data.Dataset class that takes a list of data examples, a path to those examples
# a data transform and outputs a torch dataset.
from functions.data.mri_dataset import SliceDataset
# Unet architecture as nn.Module
from functions.models.unet import Unet
# Function that returns a MaskFunc object either for generatig random or equispaced masks
from functions.data.subsample import create_mask_for_mask_type
# Implementation of SSIMLoss
from functions.training.losses import SSIMLoss

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

SEED = 1
# seed
random.seed(SEED)
np.random.seed(SEED)
torch.cuda.manual_seed(SEED)
torch.manual_seed(SEED)


Data

In [2]:
path_data1 = '/cheng/metaMRI/metaMRI/data_dict/E7.1/brain_train_FLAIR_Skyra_5-8.yaml'
path_data2 = '/cheng/metaMRI/metaMRI/data_dict/E7.1/brain_train_FLAIR_Prisma_5-8.yaml'


# hyperparameter
target_shot = 150

# mask function and data transform
mask_function = create_mask_for_mask_type(mask_type_str = 'random', self_sup = False, 
                    center_fraction = 0.08, acceleration = 4.0, acceleration_total = 3.0)

# fix mask
data_transform = UnetDataTransform_norm('multicoil', mask_func = mask_function, use_seed=True, mode='adapt')

target_set1 = SliceDataset(dataset = path_data1, path_to_dataset='', path_to_sensmaps=None, provide_senmaps=False, 
                        challenge="multicoil", transform=data_transform, use_dataset_cache=True, num_samples = target_shot)
target_set2 = SliceDataset(dataset = path_data2, path_to_dataset='', path_to_sensmaps=None, provide_senmaps=False, 
                        challenge="multicoil", transform=data_transform, use_dataset_cache=True, num_samples = target_shot)

dataloader1 = torch.utils.data.DataLoader(target_set1, batch_size = 1, num_workers = 8, 
                        generator = torch.Generator().manual_seed(1), pin_memory = True)
dataloader2 = torch.utils.data.DataLoader(target_set2, batch_size = 1, num_workers = 8, 
                        generator = torch.Generator().manual_seed(1), pin_memory = True)


flattened_target_set1 = []
for images in dataloader1:
    input_image, target_image, mean, std, fname, slice_num = images
    flattened_input = torch.flatten(input_image, start_dim=1, end_dim=3)
    flattened_target_set1.extend(flattened_input)
flattened_target_set2 = []
for images in dataloader2:
    input_image, target_image, mean, std, fname, slice_num = images
    flattened_input = torch.flatten(input_image, start_dim=1, end_dim=3)
    flattened_target_set2.extend(flattened_input)


flattened_target_set1 = torch.stack(flattened_target_set1)
flattened_target_set2 = torch.stack(flattened_target_set2)
print('Target data1 shape: ', flattened_target_set1.shape)
print('Target data2 shape: ', flattened_target_set2.shape)

# convert to numpy
target_1 = flattened_target_set1.numpy()
target_2 = flattened_target_set2.numpy()

Target data1 shape:  torch.Size([150, 102400])
Target data2 shape:  torch.Size([150, 102400])


Label

In [3]:
Y = [0] * target_shot
for i in range(target_shot+1, target_shot*2+1):
    Y.append(1)

Classification

In [4]:
from sklearn.decomposition import PCA 
# concat the data
flattened_target_set = np.concatenate((flattened_target_set1, flattened_target_set2), axis=0)
# dimension reduction evaluation
N = 100
pca = PCA(n_components=N).fit(flattened_target_set) 
pca_ratio = pca.explained_variance_ratio_
np.sum(pca_ratio[0:N-1])

0.9034733

In [5]:
target_pca = PCA(n_components=N).fit_transform(flattened_target_set) 
target_pca.shape

(300, 100)

In [6]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0).fit(target_pca)
kmeans.labels_

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


array([1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1], d

In [7]:
matching_elements = np.equal(kmeans.labels_, Y)
matching_ratio = np.sum(matching_elements) / matching_elements.size
print("Matching ratio: {:.2f}%".format(matching_ratio * 100))

Matching ratio: 53.00%


In [8]:
from sklearn.manifold import TSNE
target_tsne = np.concatenate((flattened_target_set1, flattened_target_set2), axis=0)
X_tsne = TSNE(n_components=3, learning_rate='auto', init='random', perplexity=30).fit_transform(target_tsne)

In [9]:
kmeans = KMeans(n_clusters=2, random_state=0).fit(X_tsne)
kmeans.labels_

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], d

In [10]:
matching_elements = np.equal(kmeans.labels_, Y)
matching_ratio = np.sum(matching_elements) / matching_elements.size
print("Matching ratio: {:.2f}%".format(matching_ratio * 100))

Matching ratio: 35.33%
